In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.merrill import Merrill
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [7]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,1).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
merrill = Merrill()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
accurate = market.retrieve_data("accurate")
relevant_tickers = list(accurate["ticker"])
market.close()
reload = True
quarterly_range = range(1,5)
yearly_range = range(2017,2021)
model_range = range(len(tickers["Symbol"]))
bonds = ["A","AAA","B","BBB"]
dataset = "pdr"

In [3]:
ds = []
if reload:
    merrill.connect()
    for bond in bonds:
        data = merrill.retrieve_data(bond)
        ds.append(data)
    merrill.close()

In [4]:
base = ds[0]
base.rename(columns={"yield":bonds[0]},inplace=True)
for i in range(1,len(bonds)):
    data = ds[i]
    data.rename(columns={"yield":bonds[i]},inplace=True)
    base = base.merge(data.drop("_id",axis=1),on="date",how="left")

In [5]:
base = DateTransformer.convert_to_date("pdr",base,"date")
base["year"] = [x.year for x in base["date"]]
base["quarter"] = [x.quarter for x in base["date"]]
base["week"] = [x.week for x in base["date"]]
bond_weekly = base.groupby(["year","quarter","week"]).mean().reset_index()

In [8]:
sims = []
gap = 7
training_years = 7
sec.connect()
strat_db.connect()
market.connect()
diff = market.retrieve_data("diff")
# strat_db.drop_table("{}_weekly_sim".format(dataset))
inaccuracy = diff[diff["diff"] > 0.1]["ticker"]
strat_db.drop_table("{}_bond_quarterly_sim".format(dataset))
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        try:
            ## Setting Up
            print(year,quarter)
            date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
            training_start,training_end,prediction_start,prediction_end = date_ranges
            dates = pd.to_datetime(date_ranges)
            quarters = [x.quarter for x in dates]
            years = [x.year for x in dates]
            print(date_ranges)
            ### switch to all tickers
            for ticker in list(accurate["ticker"]):
                try:
                    price = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                    if dataset == "pdr":
                        price = ColumnTransformer.rename_columns(price," ")
                    else:
                        price = ColumnTransformer.rename_columns(price,"")
                    price = DateTransformer.convert_to_date(dataset,price,"date")
                    mt = ModelTransformer(ticker,training_start,training_end,gap)
                    pt = ModelTransformer(ticker,prediction_start,prediction_end,gap)
                    mr = ModelPreprocessor(ticker)
                    prot = ProductTransformer(ticker,prediction_start,prediction_end)
                    ## regression_model
                    rfd = mt.bond_merge(price.copy(),bond_weekly.copy(),True,classify=False).reset_index()
                    refined = mr.fundamental_preprocess(rfd.copy())
                    rfr = sp.regression(refined,ranked=False,tf=False,deep=False)
                    # classification_model
                    cfd = mt.bond_merge(price.copy(),bond_weekly.copy(),True,classify=True).reset_index()
                    refined = mr.fundamental_preprocess(cfd.copy())
                    cfr = sp.classification(refined,tf=False,deep=False)
                    fundamental_results = pd.DataFrame([cfr,rfr])
                    product_refineds = []
                    product_qpds = []
                    for i in range(len(fundamental_results)):
                        fundamental_result = fundamental_results.iloc[i]
                        if fundamental_result["model_type"] == "regression":
                            fundamental_data = pt.bond_merge(price.copy(),bond_weekly.copy(),True,classify=False).reset_index()
#                                 fundamental_data = fundamental_data[(fundamental_data["year"] == year) & (fundamental_data["quarter"] == quarter)]
                        else:
                            fundamental_data = pt.bond_merge(price.copy(),bond_weekly.copy(),True,classify=True).reset_index()
#                                 fundamental_data = fundamental_data[(fundamental_data["year"] == year) & (fundamental_data["quarter"] == quarter)]
                        product_refined = mr.fundamental_preprocess_rec(fundamental_data.copy())
                        product_refineds.append(product_refined)
                    sim = prot.merge_weeklies_v2(product_refineds,fundamental_results,year,quarter,"bond")
                    strat_db.store_data("{}_bond_weekly_sim".format(dataset),pd.DataFrame(sim))
                except Exception as e:
                    message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

2017 1
['2010-01-01', '2017-01-02', '2017-01-02', '2017-03-31']
{'status': 'quarterly modeling', 'ticker': 'KEYS', 'year': '2017', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'INFO', 'year': '2017', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}



 25%|█████████████████████████████████████████████▌                                                                                                                                        | 1/4 [01:39<04:57, 99.15s/it]

2017 2
['2010-04-01', '2017-03-31', '2017-04-03', '2017-06-30']
{'status': 'quarterly modeling', 'ticker': 'KEYS', 'year': '2017', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'INFO', 'year': '2017', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}



 50%|███████████████████████████████████████████████████████████████████████████████████████████                                                                                           | 2/4 [03:18<03:18, 99.09s/it]

...

{'status': 'quarterly modeling', 'ticker': 'BSX', 'year': '2020', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
2020 3
['2013-07-01', '2020-06-30', '2020-07-01', '2020-09-30']
{'status': 'quarterly modeling', 'ticker': 'BRK-B', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'LH', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AZO', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'HSIC', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'SNPS', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'FISV', 'year': '2020', 'quarter': '3', 'message': 'sing


 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                             | 3/4 [01:47<00:35, 36.00s/it]

{'status': 'quarterly modeling', 'ticker': 'BSX', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-bounds'}
2020 4
['2013-10-01', '2020-09-30', '2020-10-01', '2020-12-31']
{'status': 'quarterly modeling', 'ticker': 'BRK-B', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'LH', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AZO', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'HSIC', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'SNPS', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'FISV', 'year': '2020', 'quarter': '4', 'message': 'sing


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [19:53<00:00, 298.30s/it]

{'status': 'quarterly modeling', 'ticker': 'BSX', 'year': '2020', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
